In [1]:
# mac475의 ipython 표준 style을 적용함
from IPython.core.display import HTML
styles = open("styles/custom.css", "r").read()
HTML( styles )

#8. Tree-Based Method
* tree-based method는 stratifying, segmentaing 통해 predictor space를 여러 region으로 구분
* 일반적으로는 predictor가 소속된 training observation region의 mean, mode (최빈값)을 사용함
* Tree-based method는 interpretation이 매우 용이
* supervised learning에 있어 prediction accuracy가 좋진 않으므로, 이후 bagging/ random forests/ boosting을 활용
    - multiple trees 생성후 하나로 combine

##8.1 The Basics of Decision Trees
* Decision Tree는 regression, classification problem 모두 활용가능
###8.1.1 Regression Trees
####Predicting Baseball Players' Salaries Using Regression Trees
* Tree는 internal node와 branch로 구성
    <img src="images/p304 figure 8.1.png" width = "45%"/>
* $R_1, R_2, R_3$는 terminal nodes, 혹은 leaves라고 함
    <img src="images/p305 figure 8.2.png" width = "65%"/>
* Tree 모형은 interpret이 매우 용이

####Prediction via Stratification of the Feature Space
* Tree building process는 다음과 같음
    1. predictor $X_1, X_2, \cdots, X_p$를 $J$개의 distinct하고 non-overlapping한 region $R_1, R_2, \cdots, R_J$로 divide
    2. region $R_J$에 포함되는 모든 observation들은 동일한 prediction result를 가진다고 본다
        - 이때, prediction result란 $R_j$에 포함되는 training ovservation들에 대한 response의 mean값임
* 이때, predictor space $R_1, R_2, \cdots, R_J$의 region은
    1. high-dimensional rectangle, 혹은 box 형태로 divide하고,
    2. 이때, 목표는 아래의 $RSS$값을 minimize하는 $R_1, \cdots, R_J$를 찾아내는 것임
        $$RSS = \sum\limits_{j=1}^J\sum\limits_{i \in R_j}(y_i - \hat{y}_{R_j})^2 \quad (where, \, \hat{y}_{R_j} : mean\,\, response\,\,for\,\,the\,\,training\,\,observations\,\,within\,\,the\,\,jth\,\,box)$$
* Region의 divide는 top-down, greedy approach로 진행
    - Greedy algorithm [https://en.wikipedia.org/wiki/Greedy_algorithm]: 각 결정의 시점마다, 가장 좋다(최적이다)고 판단하는 것을 해답으로 우선 선택. 단, local 시점 최적이라 생각했던 판단이 최종적인 (globally)한 최적을 보장하지는 않음
* predictor $X_j$, cutpoint $s$에 대해서, $RSS$가 minimize되어야 함. 즉,
    
    $$R_1(j,s) = \{X|X_j < s\} \,\, and \,\, R_2(j,s) = \{X|X_j \ge s\}$$
    
    $$\sum\limits_{i:x_i \in R_1(j,s)}(y_i-\hat{y}_{R_1})^2 + \sum\limits_{i:x_i \in R_2(j,s)}(y_i-\hat{y}_{R_2})^2$$
    - 이때, $\hat{y}_{R_1}$ : $R_1(j,s)$내의 training observation들에 대한 mean response이고,
    - $\hat{y}_{R_2}$ : $R_2(j,s)$내의 training observation들에 대한 mean response임
* 이러한 process를 거쳐 $R_1, \cdots, R_J$가 생성되면, test observation에 대한 prediction값은, 해당 test observation이 소속된 region내 training observation의 mean값임
    <img src="images/p308 figure 8.3.png" width = "55%"/>

####Tree Pruning (가지치기)
* Tree overfitting 방지위해, smaller tree를 통해 complexity를 줄여 bias 희생통해 lower variance를 확보가능
    - 일정 threshold 초과하는 RSS의 감소때까지만 tree building을 지속하는 방법통해 가능 → tree 개수가 줄어든다
    - 하지만, 나중에 형성될 split에서 좋은 performance가 발생가능하므로, 근시안적인 방법임
* 따라서, 더 나은 strategy는, 일단 very large tree $T_0$를 만든 후, pruning (가지치기) 함
    - 이를 Cost complexity pruning, 혹은 weakest link pruning이라고 함
    $$\sum\limits_{m=1}^{|T|}\sum\limits_{x_i \in R_m}(y_i-\hat{y}_{R_m})^2+\alpha|T|$$
    - 이때, lasso때와 같이 tuning parameter $\alpha$를 통해, complexity와 training data에 대한 fitting을 조절
        - $\alpha$가 증가할수록, branch는 가지치기됨 (이때 $\alpha$는 CV를 통해 선정)

###8.1.2 Classification Trees
* Classification tree는 regression tree와 유사하나 prediction 대상이 qualitative response임
    - Regression tree : predicted response는 observation이 소속된 region의 mean response
    - Classification tree : predicted response는 observation이 소속된 region에서 가장 빈도수가 많은 class
        - 최빈 class뿐 아니라, class proportion도 관심을 갖는다
* Regression에서는 RSS를 활용하여 그 품질을 판단가능하나, Classification에서는 *classification error rate* 활용
    $$E = 1 - \max_k(\hat{p}_{mk})$$
    - 이때, $\hat{p}_{mk}$는 $m$th region내의 $k$th class의 proportion임
* 하지만, tree-growing에 있어 classification error rate이 충분히 민감하지 않아 다음 두가지 measure가 주로 활용
    - Gini index : $\hat{p}_{mk}$가 0, 혹은 1에 접근할 수록, G값은 작아짐 → $\therefore$ node purity라고도 함
        $$G = \sum\limits_{k=1}^K\hat{p}_{mk}(1-\hat{p}_{mk})$$
        - 작은 G값은 해당 node가 single class에 dominent하다는 것을 의미
    - cross-entropy
        $$D = -\sum\limits_{k=1}^K\hat{p}_{mk}\log\hat{p}_{mk}$$
        - 이때, $0 \le \hat{p}_{mk} \le 1$이므로, $0 \le -\hat{p}_{mk}log\hat{p}_{mk}$임
        - cross-entropy 역시 Gini index와 같이 purity가 높을수록, 작은값을 가짐
    - 참고
        - 엔트로피 : 무질서량
        - 통계학적 상태분포의 엔트로피 : $S = -k_B\sum\limits_ip_i\ln(p_i)$ : cross-entropy와 유사공식
        <img src="images/p312 log graph.png" width = "45%"/>

* Classification tree의 building시 *Gini index, cross-entropy*가 특정 split의 quality 판단에 주로 활용
    - 하지만, prediction accuracy가 중요하다면, classification error가 선호됨
* 요약하면, classification tree에서는 Gini, cross-entropy값이 작아지도록 하는 split 형성을 유도하게 되며
* 이는 의미적으로 보아, 해당 split을 통해 각 node들이 pure해지는 것을 방향성과 같은 의미임
    - 각 node가 pure해진다? → 각 node가 classification되었다는 의미임

###8.1.3 Trees Versus Linear Models
* linear regression은 다음과 같은 형태
    $$f(X) = \beta_0 + \sum\limits_{j=1}^pX_j\beta_j$$
* regression tree는 다음과 같은 형태
    $$f(X) = \sum\limits_{m=1}^Mc_m\cdot1_{(X \in R_m)}$$
* linear와 tree는 problem의 형태에 따라 우위/ 열위가 나뉨 : 아래를 참고
    <img src="images/p315 figure 8.7.png" width = "65%"/>

###8.1.4 Advantages and Disadvantages of Trees
* 장점
    - 설명이 쉬움
    - 인간의 의사결정 구조와 유사함
    - graphically 표현이 가능하고 interpret 용이
    - qualitative predictor 다루기 용이
* 단점
    - tree는 일반적으로 다른 regression/ classification에 비해 predictive accuracy가 나쁨
* 대안
   - *bagging, random forests, boosting*과 같이 여러 decision tree를 aggregation하여 predictive performance 향상 가능

##8.2 Bagging, Random Forests, Boosting
* Bagging, random forests, boosting은 powerful model prediction위해 tree들을 building block형태로 활용
###8.2.1 Bagging
* Bootstrap은 dataset에 대한 standard deviation 등을 직접계산이 불가능한 경우 유용하게 사용되는 방법
* 하지만, statistical learning method를 향상시키는 역할수행도 가능
* **B**ootstrap **agg**regation, or **Bagg**ing : variance를 낮춰주는 역할수행
    - 각각의 variance가 $\sigma^2$인 $Z_1, \cdots, Z_n$ 존재시, mean $\bar{Z}$의 variance는 $\sigma^2/n$임 (증명하진 못했음)
    - 즉, observation의 averaging통해 variance를 reduce함으로써 statistical learning method의 accuracy를 향상가능
        - 따라서, 모집단으로부터 많은 training set를 취하여 각각의 predict model을 만든후 averaging result라는 의미임
    $$\hat{f}_{avg}(x) = \frac{1}{B}\sum\limits_{b=1}^B\hat{f}_b(x)$$
    - 단, 일반적으로 training set 여러개인 경우 없으므로, bootstrap 통해 training set에서 여러개의 sample 취하게 됨
        - B개의 서로다른 bootstrapped training sets를 생성
    $$\hat{f}_{bag}(x) = \frac{1}{B}\sum\limits_{b=1}^B\hat{f}_b(x)$$
* Bagging 통해 (not pruned) deep tree를 여러개 확보하게 되며, 이때 deep tree들은 low bias, high variance이고, 여러개의 deep tree들을 averaging/ voting 통해 variance를 낮춤으로써, predict accuracy를 높이게 됨
    - regression은 average, classification은 voting

####Out-of-Bag Error Estimation
* Bagging시 bootstrap 활용하게 되며, bootstrap에 대해
    - n개 observation을 가지는 집합에서 i번째의 observation이 bootstrap sample이 아닐 확률은 (중복을 허용하므로...)
    $$P(out) = (1-1/n)^n$$
    - 따라서, bootstrap sample일 확률은
    $$P(in) = 1 - P(out) = 1 - (1-1/n)^n$$
    - 이고, 각 $n=5, n=100, n=10000$에 대한 결과값을 보면, 다음과 같이 63%에 수렴
    $$P(in) = 1 - (1-1/5)^5 = 67.2\%\\=1-(1-1/100)^{100} = 63.4\%\\=1-(1-10000)^{10000} = 63.2\%$$
    - 따라서, bagged tree는 1/3정도 observation 활용, 나머지 1/3을 **OOB (out-of-bag)** observation이라 함
        - 이때, bootstarp시 만든 여러개의 model로 OOB에 대해 각각 predict 가능하며, averge 계산통해 single predict도 가능
        - 이를 통해, OOB MSE (for regression), OOB classification error (for classification) 계산가능

####Variable Importance Measures
* Bagging은 accuracy를 향상시키지만, interpret이 곤란
* 하지만, RSS, 혹은 Gini index 활용하여 각 predictor의 importance 계산가능

###8.2.2 Random Forests
* Random Forests는 생성되는 여러 model을 서로 decorrelate시키는 방법으로, Bagging 비해 향상된 결과 제공
* Random Forests는 Bagging과 진행절차가 유사하며,
    1. 여러개의 Bootstrapped training sample을 기반으로,
    2. 여러개의 decision tree를 생성하는데 있어,
    3. tree building시, split 생성마다 (총 $p$ predictor중) random $m$개 predictor를 split candidate로 선택
    4. 이때, 각 split 생성시 고려되는 $m$은 일반적으로 $m \approx \sqrt{p}$임
* 가령, predictor중 특히 강한 중요도 가지는 것 존재시, Bagging이 bootstrap 기반이라도, 생성되는 model들은 서로 유사형태일 것이므로 - 즉, highly correlated - 이를 averaging해도 variance reduction이 낮음
    - Random Forests는 이 문제점을 split 생성시점 고려 predictor를 subset화하여 극복 : decorrelating the trees
    - 일반적으로는 Bagging/ Random Forests 모두 tree의 개수가 증가함에 따라 overfit을 극복가능/ error rate 하락
    <img src="images/p321 figure 8.10.png" width = "80%"/>

###8.2.3 Boosting
* Boosting은 decision tree의 향상방법
* Boosting은 bagging과 유사하게 진행되나,
    - Weak tree가 sequential하게 growing (이때, 이전 tree의 정보를 활용하여)
    - tree가 growing된 후, data는 reweighted됨
        - 잘못분류된 data : weight 상승
        - 정확히 분류된 data : weight 하락
        - 따라서, future learner는 이전 tree가 잘못분류한 것들에 focus하게됨
    - Bootstrap을 사용하지 않고, 전체 dataset 활용
* Bagging이 여러 model을 한번에 averaging하여 fitting the data hard하는 반면, Boosting은 learns slowly임
    - Boosting은 이전에 만들어진 tree에 의존하여 growing함
* Boosting은 tuning parameter는
    1. Tree 개수
    2. shrinkage parameter $\lambda$ : 일반적으로 (0.01, 0.001)이며, boosting learning rate을 조절
    3. number $d$ of splits in each tree
* 수행 알고리즘은 다음과 같음 : fitting시 response $Y$가 아니라, residual $r$에 수행
    <img src="images/p323 algorithm 8.2.png" width = "65%"/>
* Bagging과 Boosting 비교요약[http://jangjy.tistory.com/166]
    <img src="images/p324 bagging flow.png" width = "80%"/>
    
    <img src="images/p324 boosting flow.png" width = "80%"/>
    
    <img src="images/p324 bagging boosting.png" width = "80%"/>